# Ping Pong Paddle Maneuvers

_Final Project for 6.843, Fall 2021_

_Viraj Parimi, Cameron Pittman_

In [1]:
import numpy as np
from manipulation.meshcat_cpp_utils import (
    StartMeshcat
)
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess

from pydrake.all import (
    AddTriad,
    BasicVector,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MeshcatVisualizerCpp,
    MeshcatVisualizerParams,
    RollPitchYaw,
    Simulator,
    MathematicalProgram, SnoptSolver
)

from iiwa_paddle_station import MakeIiwaPaddleStation

proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

In [2]:
meshcat = StartMeshcat()

In [3]:
class PseudoInverseController(LeafSystem):
    """
    Based on the PseudoInverseController from pset3/robot_painter
    """

    def __init__(self, plant):
        LeafSystem.__init__(self)
        self.plant = plant
        self.plant_context = plant.CreateDefaultContext()
        self.iiwa = plant.GetModelInstanceByName("iiwa7")
        self.P = plant.GetBodyByName("base_link").body_frame()
        self.W = plant.world_frame()

        self.q_port = self.DeclareVectorInputPort("iiwa_pos_measured", BasicVector(7))
        self.w_P_port = self.DeclareVectorInputPort(
            "paddle_desired_angular_velocity", BasicVector(3)
        )
        self.v_P_port = self.DeclareVectorInputPort(
            "paddle_desired_velocity", BasicVector(3)
        )

        self.DeclareVectorOutputPort("iiwa_velocity", BasicVector(7), self.CalcOutput)

        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context, output):
        q = self.q_port.Eval(context)
        w_P = np.zeros(3)
        # w_P = self.w_P_port.Eval(context)
        v_P = self.v_P_port.Eval(context)
        self.plant.SetPositions(self.plant_context, self.iiwa, q)
        J_P = self.plant.CalcJacobianSpatialVelocity(
            self.plant_context,
            JacobianWrtVariable.kV,
            self.P,
            [0, 0, 0],
            self.W,
            self.W,
        )
        J_P = J_P[:, self.iiwa_start : self.iiwa_end + 1]  # Only iiwa terms.
        V_P = np.hstack([w_P, v_P])
        v = np.linalg.pinv(J_P).dot(V_P)  # important
        output.SetFromVector(v)


In [4]:
class DifferentialIKSystem(LeafSystem):
    """Wrapper system for Differential IK. 
        @param plant MultibodyPlant of the simulated plant. 
        @param diffik_fun function object that handles diffik. Must have the signature 
               diffik_fun(J_G, V_G_desired, q_now, v_now, X_now)
    """ 
    def __init__(self, plant, diffik_fun):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa7")
        self._G = plant.GetBodyByName("base_link").body_frame()
        self._W = plant.world_frame()
        self._diffik_fun = diffik_fun

        self.DeclareVectorInputPort("spatial_velocity", BasicVector(6))
        self.DeclareVectorInputPort("iiwa_position_measured", BasicVector(7))
        self.DeclareVectorInputPort("iiwa_velocity_measured", BasicVector(7))
        self.DeclareVectorOutputPort("iiwa_velocity_command", BasicVector(7), 
                                     self.CalcOutput)
        
    def CalcOutput(self, context, output):
        V_G_desired = self.get_input_port(0).Eval(context)
        q_now = self.get_input_port(1).Eval(context)
        v_now = self.get_input_port(2).Eval(context)

        self._plant.SetPositions(self._plant_context, self._iiwa, q_now)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kQDot, 
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,0:7] # Ignore gripper terms

        X_now = self._plant.CalcRelativeTransform(self._plant_context, 
                                                     self._W, self._G)
        p_now = X_now.translation()
        rpy_now = RollPitchYaw(X_now.rotation()).vector()
        
        v = self._diffik_fun(J_G, V_G_desired, q_now, v_now, p_now, rpy_now) 
        output.SetFromVector(v)

In [5]:
def deg_to_rad(deg):
  return (deg / 360) * 2 * np.pi

def DiffIKQP(J_G, V_G_desired, q_now, v_now, p_now, rpy_now):

  prog = MathematicalProgram()
  v = prog.NewContinuousVariables(7, 'v')
  v_max = 3.0 # do not modify
  h = 4e-3 # do not modify

  obj = J_G.dot(v) - V_G_desired
  prog.AddCost(obj.dot(obj))
  prog.AddBoundingBoxConstraint(-v_max * np.ones(7), v_max * np.ones(7), v)

  solver = SnoptSolver()
  result = solver.Solve(prog)

  if not (result.is_success()):
    raise ValueError("Could not find the optimal solution.")

  v_solution = result.GetSolution(v)

  return v_solution

In [16]:
def ball_p_from_state(pose):
    """
    Get x, y, z from ball_state

    Params:
        BasicVector(13)
    Returns:
        array[3]
    """
    return pose[4:7]


def ball_v_from_state(pose):
    """
    Get translational velocity from ball_state

    Params:
        BasicVector(13)
    Returns:
        array[3]
    """
    return pose[-3:]


# TODO: could we use a signed distance function instead?
def get_mode(p_ball, v_ball, p_paddle, atol = 1e-03):
    """
    Determine current paddle mode

    Output is one of: "rising", "falling", "contact"
    """
    b_z = p_ball[2]
    vz = v_ball[2]
    p_z = p_paddle[2]

    # TODO: there's a better way to do this
    if np.isclose(b_z, p_z, atol=atol):
        return "contact"
    else:
        return "rising" if vz >= 0 else "falling"


delta_height = 0.2
CATCH_HEIGHT = 0.55
g = -9.81
# g = -3.721 # mars
e_z = np.array([0, 0, 1], dtype = float)

# hardcoded initial paddle position
p_p_0 = np.array([0.07659419, -0.75730758, 0.63302061])

# initial x, y, z of ball
p_b_0 = p_p_0 + np.array([-0.2, -0.1, 1])

# initial vx, vy, vz of ball
v_b_0 = np.array([0.1, 0, 0])

class Catcher5(LeafSystem):
    """
    Catch the ball
    """

    def __init__(self, plant):
        LeafSystem.__init__(self)
        self.plant = plant
        self.plant_context = plant.CreateDefaultContext()
        self.iiwa = plant.GetModelInstanceByName("iiwa7")
        self.P = plant.GetBodyByName("base_link")
        self.ball = plant.GetBodyByName("ball")
        self.W = plant.world_frame()
        
        self.t_catch = None
        # assume falling at first
        self.made_contact = False

        self.DeclareVectorInputPort("ball_pose", BasicVector(3))
        self.DeclareVectorInputPort("ball_velocity", BasicVector(3))
        self.DeclareVectorInputPort("iiwa_pos_measured", BasicVector(7))
        self.DeclareVectorInputPort("iiwa_velocity_estimated", BasicVector(7))
        self.DeclareVectorInputPort("ball_state", BasicVector(13))
        self.DeclareVectorOutputPort(
            "paddle_velocity_desired", BasicVector(6), self.CalcOutput
        )

    def compute_time_of_impact(self, p_bz, v_bz, catch_height):
        g_pos = abs(g)
        return (1. / g_pos) * (v_bz + np.sqrt(v_bz**2 + 2 * g_pos * (p_bz - catch_height)))

    def CalcOutput(self, context, output):
        q = self.GetInputPort("iiwa_pos_measured").Eval(context)
        v = self.GetInputPort("iiwa_velocity_estimated").Eval(context)
        # self.plant.SetPositions(self.plant_context, self.iiwa, q)
        self.plant.SetPositionsAndVelocities(self.plant_context, self.iiwa, np.hstack([q, v]))
        v = self.GetInputPort("iiwa_velocity_estimated").Eval(context)

        p_Ball = self.GetInputPort("ball_pose").Eval(context)
        v_Ball = np.array(self.GetInputPort("ball_velocity").Eval(context))

        p_Paddle = self.plant.EvalBodyPoseInWorld(
            self.plant_context, self.P
        ).translation()

        v_Paddle = self.plant.EvalBodySpatialVelocityInWorld(
            self.plant_context, self.P
        ).translational()

        v_p_desired = np.zeros(3)
        w_p_desired = np.zeros(3)

        current_time = context.get_time()

        mode = get_mode(p_Ball, v_Ball, p_Paddle, atol=1e-01)
        if not self.made_contact and mode == "contact":
            self.made_contact = True

        max_angle = 30
        paddle_radius = 0.175 # according to the paddle.sdf
        ball_radius = 0.035 # according to ball.sdf
        gain = np.array([-35, 35, 0])

        # useful when trying to match the positions of the ball and paddle
        dist_ball_and_paddle = p_Ball - p_Paddle - np.array([0, 0, ball_radius])

        [x_offset, y_offset, z_offset] = (dist_ball_and_paddle) / paddle_radius
        
        w_p_x = np.tan(y_offset * max_angle)
        w_p_y = np.tan(x_offset * max_angle) * -1
        w_p_z = 0

        w_p_desired = np.array(list(map(deg_to_rad, [w_p_x, w_p_y, w_p_z]))) * gain

        if not self.made_contact:
            # try to beat the ball to the catch point
            if self.t_catch is None:
                self.t_catch = self.compute_time_of_impact(p_Ball[2], v_Ball[2], CATCH_HEIGHT)

            # tell the iiwa to get to the location of the catch a split second early
            t_pre_catch = self.t_catch * 0.75

            # assume ball is only acting under gravity
            a_b = np.array([0, 0, g])

            p_catch = p_b_0 + v_b_0 * self.t_catch + (a_b / 2) * self.t_catch**2
            v_b_catch = v_b_0 + a_b * self.t_catch

            v_p_0 = (p_catch - p_p_0 - 0.5 * v_b_catch * t_pre_catch) / (0.5 * t_pre_catch)
            a_p = (v_b_catch - v_p_0) / t_pre_catch

            v_p_desired = v_p_0 + a_p * current_time
        else:
            kp = 10
            kd = 0.1
            # ball should be moving slowly. change to PD behavior after the intial contact
            v_p_desired = kp * dist_ball_and_paddle + kd * (v_Ball - v_Paddle)

        V_P = np.concatenate([w_p_desired, v_p_desired])
        output.SetFromVector(V_P)


In [17]:
class Demo:
    def __init__(self, time_step=0.001):
        """
        Robotic Kuka iiwa with paddle end effector

        Args:
            time_step (float, optional): time step for internal manipulation station controller
        """
        self.time = 0
        diagram, plant, scene_graph = MakeIiwaPaddleStation(time_step=time_step)

        self.builder = DiagramBuilder()
        self.station = self.builder.AddSystem(diagram)
        self.position_log = []
        self.velocity_log = []

        self.visualizer = MeshcatVisualizerCpp.AddToBuilder(
            self.builder,
            diagram.GetOutputPort("geometry_query"),
            meshcat,
            MeshcatVisualizerParams(delete_prefix_initialization_event=False)
        )

        # change gravity
        plant.mutable_gravity_field().set_gravity_vector([0, 0, g])

        self.plant = plant

        diff_ik = self.builder.AddSystem(DifferentialIKSystem(self.plant, DiffIKQP))
        diff_ik.set_name("diff_ik_sys")
        catcher = self.builder.AddSystem(Catcher5(self.plant))
        catcher.set_name("catcher")
        integrator = self.builder.AddSystem(Integrator(7))


        self.builder.Connect(diff_ik.get_output_port(),
                        integrator.get_input_port())
        self.builder.Connect(integrator.get_output_port(),
                        self.station.GetInputPort("iiwa_position"))
        self.builder.Connect(self.station.GetOutputPort("iiwa_position_measured"),
                        diff_ik.get_input_port(1))
        self.builder.Connect(self.station.GetOutputPort("iiwa_velocity_estimated"),
                        diff_ik.get_input_port(2))
        self.builder.Connect(catcher.get_output_port(0),
                        diff_ik.get_input_port(0))

        self.builder.Connect(self.station.GetOutputPort("iiwa_position_measured"), catcher.GetInputPort("iiwa_pos_measured"))
        self.builder.Connect(self.station.GetOutputPort("iiwa_velocity_estimated"), catcher.GetInputPort("iiwa_velocity_estimated"))

        self.builder.Connect(
            self.station.GetOutputPort("ball_state"),
            catcher.GetInputPort("ball_state")
        )

        self.builder.ExportInput(catcher.GetInputPort("ball_pose"), "v_ball_pose")
        self.builder.ExportInput(catcher.GetInputPort("ball_velocity"), "v_ball_velocity")

        self.diagram = self.builder.Build()
        self.simulator = Simulator(self.diagram)
        self.simulator.set_target_realtime_rate(1.0)

        self.context = self.simulator.get_context()
        self.station_context = self.station.GetMyContextFromRoot(self.context)
        self.plant_context = self.plant.GetMyContextFromRoot(self.context)

        iiwa_q0 = [-1.57, 0, 0, -np.pi/2, 0, 0, 0]
        self.plant.SetPositions(self.plant_context, self.plant.GetModelInstanceByName("iiwa7"), np.array(iiwa_q0))

        # [<4 quaternion>, <3 translational pos>, <3 angular vel>, <3 translational vel>]
        ball_q0 = np.concatenate([ np.ones(4), p_b_0, np.zeros(3), v_b_0 ])
        self.plant.SetPositionsAndVelocities(self.plant_context, self.plant.GetModelInstanceByName("ball"), np.array(ball_q0))

        self.station.GetInputPort("iiwa_feedforward_torque").FixValue(self.station_context, np.zeros((7,1)))
        iiwa_model_instance = self.plant.GetModelInstanceByName("iiwa7")
        iiwa_q = self.plant.GetPositions(self.plant_context, iiwa_model_instance)
        integrator.GetMyContextFromRoot(self.context).get_mutable_continuous_state_vector().SetFromVector(iiwa_q)


    # borrowed from github.com/matt-bev/robot-juggler
    def step(self, simulate=True, duration=0.1, final=True, verbose=False, display_pose=False):
        """
        step the closed loop system

        Args:
            simulate (bool, optional): whether or not to visualize the command. Defaults to True.
            duration (float, optional): duration to complete command in simulation. Defaults to 0.1.
            final (bool, optional): whether or not this is the final command in the sequence; relevant for recording. Defaults to True.
            verbose (bool, optional): whether or not to print measured position change. Defaults to False.
            display_pose (bool, optional): whether or not to show the pose of the paddle in simulation. Defaults to False.
        """
        ball_pose = self.plant.EvalBodyPoseInWorld(self.plant_context, self.plant.GetBodyByName("ball")).translation()
        ball_velocity = self.plant.EvalBodySpatialVelocityInWorld(self.plant_context, self.plant.GetBodyByName("ball")).translational()
        self.diagram.GetInputPort("v_ball_pose").FixValue(self.context, ball_pose)
        self.diagram.GetInputPort("v_ball_velocity").FixValue(self.context, ball_velocity)

        if display_pose:
            transform = self.plant.EvalBodyPoseInWorld(self.plant_context, self.plant.GetBodyByName("base_link")).GetAsMatrix4()
            AddTriad(self.visualizer.vis, name=f"paddle_{round(self.time, 1)}", prefix='', length=0.15, radius=.006)
            self.visualizer.vis[''][f"paddle_{round(self.time, 1)}"].set_transform(transform)

        if simulate:

            self.visualizer.StartRecording()
            self.simulator.AdvanceTo(self.time + duration)
            self.visualizer.StopRecording()

            self.position_log.append(self.station.GetOutputPort("iiwa_position_measured").Eval(self.station_context))
            self.velocity_log.append(self.station.GetOutputPort("iiwa_velocity_estimated").Eval(self.station_context))

            if verbose:
                print("Time: {}\nMeasured Position: {}\n\n".format(round(self.time, 3), np.around(self.station.GetOutputPort("iiwa_position_measured").Eval(self.station_context), 3)))

            if final:
                self.visualizer.PublishRecording()
                return self.time + duration

        self.time += duration


time_step = .001
demo = Demo(time_step=time_step)


seconds = 2
for i in range(int(seconds*20)):
    t = demo.step(duration=0.05, final=i==seconds*20-1, verbose=False, display_pose=False)
    if t is not None and t != seconds:
        break

In [ ]:
juggler.visualizer.PublishRecording()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b95b60bf-1e32-44f8-a07c-9240db62d20a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>